In [6]:
import os
import anndata
import pandas as pd
import anndata as ad
from FlowCytometryTools import FCMeasurement
import re
import numpy as np

### Create functions to load and concatenate data

In [7]:
def arcsinh_transform(X,cofactor=5):
    return np.arcsinh(X/cofactor)

In [8]:
def load_data_fcs(path,condition_name):
    data = FCMeasurement(ID="Sample", datafile=path)
    anndata = ad.AnnData(data.data)
    
    pattern = r"_(\w+)\s*-\s*(\w+)\.fcs$"
    match_stim = re.search(pattern, path)
    if match_stim:
        #stim_type = match_stim.group(1)
        cell_type = match_stim.group(2) 
        anndata.obs['drug']= condition_name
        anndata.obs['cell_type']= cell_type
    else:
        print('No cell_type found, bad format')
    anndata.X=arcsinh_transform(anndata.X)
    return anndata

In [9]:
def concatenate_2conditions_data(path_stim,stim_name,path_unstim,unstim_name,outdir_path):
    unstim_anndata = load_data_fcs(path_unstim,unstim_name)
    stim_anndata = load_data_fcs(path_stim,stim_name)

    combined_anndata = stim_anndata.concatenate(unstim_anndata, batch_key="condition", batch_categories=["stim", "control"])
    combined_anndata.write(outdir_path)
    return

In [10]:
def concatenate_2conditions_multiple_data(path_stim_list,stim_name,path_unstim_list,unstim_name,outdir_path):
    comb_unstim_anndata = load_data_fcs(path_unstim_list[0],unstim_name)
    comb_stim_anndata = load_data_fcs(path_stim_list[0],stim_name)
    compt_un=1
    compt_st=1
    for path_stim,path_unstim in zip(path_stim_list[1:],path_unstim_list[1:]):
        unstim_anndata = load_data_fcs(path_unstim,unstim_name)
        stim_anndata = load_data_fcs(path_stim,stim_name)

        comb_unstim_anndata = comb_unstim_anndata.concatenate(unstim_anndata)
        comb_stim_anndata = comb_stim_anndata.concatenate(stim_anndata)
        compt_un+=1
        compt_st+=1
        print(compt_un,compt_st)
    combined_anndata=comb_stim_anndata.concatenate(comb_unstim_anndata, batch_key="condition", batch_categories=["stim", "control"])
    
    combined_anndata.write(outdir_path)
    return 

In [11]:
def concatenate_1condition_multiple_data(path_list,condition_name,outdir_path):
    comb_anndata = load_data_fcs(path_list[0],condition_name)

    for path in path_list[1:]:
        new_anndata = load_data_fcs(path,condition_name)
        comb_anndata = comb_anndata.concatenate(new_anndata)
    comb_anndata.write(outdir_path)
    return 

In [12]:
def create_list_of_paths(directory,stimulation,cell_type=None,sample=None,patient_excluded=None):
    paths_list = []
    for filename in os.listdir(directory):
        if stimulation in filename and (cell_type is None or cell_type in filename) and (sample is None or sample in filename) and (patient_excluded is None or patient_excluded not in filename):
            paths_list.append(os.path.join(directory, filename))
    return paths_list


In [13]:
def create_list_of_paths_spec_patients(directory,stimulation,cell_type=None,sample=None,patient=None):
    paths_list = []
    for filename in os.listdir(directory):
        if stimulation in filename and (cell_type is None or cell_type in filename) and (sample is None or sample in filename) and (patient is None or patient in filename):
            paths_list.append(os.path.join(directory, filename))
    return paths_list

###  Examples

In [10]:
path_unstim='datasets/atest_data/controls_1_Surge_3_HCAA_BL_Unstim - Bcells.fcs'
path_stim = "datasets/atest_data/controls_1_Surge_3_IC1_IC1_Stim_1 - Bcells.fcs"
outdir_path = "datasets/atest_data/combined_Bcell_norm.h5ad"
concatenate_2conditions_data(path_stim,'Stim_1_Bcell', path_unstim,'Unstim_Bcell', outdir_path)
combined_anndata=ad.read("datasets/atest_data/combined_Bcell_norm.h5ad")

/Users/MacBook/miniconda3/envs/cellot_env/lib/python3.8/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
... storing 'drug' as categorical
... storing 'cell_type' as categorical


In [11]:
directory = '/Users/MacBook/stanford/cellot/datasets/surge_prehab_controls - Gated Populations'
stimulations = ['LPS','Unstim']
def count_files(directory,stimulation):
    count = 0
    sumsize = 0
    for filename in os.listdir(directory):
        if filename.endswith(".fcs") and stimulation in filename:
            count+=1
            sumsize+=os.path.getsize(os.path.join(directory, filename))
    return count,sumsize
for stimulation in stimulations:
    print(stimulation,count_files(directory,stimulation))

LPS (126, 849626243)
Unstim (133, 1048400869)


In [12]:
path_unstim_list=create_list_of_paths(directory='/Users/MacBook/stanford/cellot/datasets/surge_prehab_controls - Gated Populations',stimulation='Unstim',cell_type='Bcells',patient_excluded='HCAA')
path_stim_list=create_list_of_paths(directory='/Users/MacBook/stanford/cellot/datasets/surge_prehab_controls - Gated Populations',stimulation='LPS',cell_type='Bcells',patient_excluded='HCAA')
res=concatenate_2conditions_multiple_data(path_stim_list,'LPS',path_unstim_list,'Unstim',"datasets/sherlock_training_data/combined_LPS.h5ad")


2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 14
15 15
16 16


... storing 'drug' as categorical
... storing 'cell_type' as categorical


In [16]:
path_unstim_HCAA=create_list_of_paths_spec_patients(directory='/Users/MacBook/stanford/cellot/datasets/surge_prehab_controls - Gated Populations',stimulation='Unstim',cell_type='Bcells',patient='HCAA')
path_stim_HCAA=create_list_of_paths_spec_patients(directory='/Users/MacBook/stanford/cellot/datasets/surge_prehab_controls - Gated Populations',stimulation='LPS',cell_type='Bcells',patient='HCAA')
res_HCAA=concatenate_2conditions_multiple_data(path_stim_HCAA,'LPS',path_unstim_HCAA,'Unstim',"/Users/MacBook/stanford/cellot/results/test_1/model-cellot/unseen_data/combined_LPS_HCAA.h5ad")

2 2


... storing 'drug' as categorical
... storing 'cell_type' as categorical


In [20]:
re=ad.read("/Users/MacBook/stanford/cellot/results/test_1/model-cellot/unseen_data/combined_LPS_HCAA.h5ad")
re.obs.head(1)

,drug,cell_type,batch,condition
0-0-stim,LPS,Bcells,0,stim


### Pipeline for evaluation

In [16]:
eval_4i=pd.read_csv("/Users/MacBook/stanford/cellot/results/4i/drug-cisplatin/model-cellot/evals_iid_data_space/evals.csv")

In [17]:
print('eval metric available:',eval_4i['metric'].unique())

eval metric available: ['l2-means' 'l2-stds' 'r2-means' 'r2-stds' 'r2-pairwise_feat_corrs'
 'l2-pairwise_feat_corrs' 'mmd' 'enrichment-k50' 'enrichment-k100']


In [18]:
eval_4i.ncells.unique()

array([100, 250, 500])

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
def create_density_plots(dist_data, out_file, title_suffix=""):
    """
    Create density plots for Unstim, Stim True, and Stim Pred distributions.
    """
    sns.set_theme(style="whitegrid")
    pts_sorted = sorted(dist_data.keys())

    fig = plt.figure(figsize=(18, 18))
    fig.suptitle(f"Density Plots {title_suffix}", fontsize=16)

    rows = int(np.ceil(len(pts_sorted) / 3))
    for i, pt in enumerate(pts_sorted, start=1):
        ax = fig.add_subplot(rows, 3, i)
        cat_labels = ["Unstim", "Stim True", "Stim Pred"]
        cat_colors = ["blue", "red", "green"]
        for label, color in zip(cat_labels, cat_colors):
            arr = dist_data[pt][label]
            if arr.size > 0:
                sns.kdeplot(arr, ax=ax, label=f"{label} (n={arr.size})", color=color, fill=False)
        ax.set_title(f"Patient: {pt}")
        ax.legend(fontsize=8)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(out_file, dpi=150)
    plt.close()

In [28]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def create_density_plots2(dist_data, out_file, title_suffix=""):
    """
    Create density plots for Unstim, Stim True, and Stim Pred distributions.
    """
    sns.set_theme(style="whitegrid")
    pts_sorted = sorted(dist_data.keys())

    num_plots = len(pts_sorted)
    cols = min(3, num_plots)  # Max 3 colonnes
    rows = int(np.ceil(num_plots / cols))  # Nombre de lignes

    fig_width = 5 * cols  # Augmenter la largeur
    fig_height = 5 * rows  # Ajustement hauteur
    fig, axes = plt.subplots(rows, cols, figsize=(fig_width, fig_height), constrained_layout=True)

    fig.suptitle(f"Density Plots {title_suffix}", fontsize=20)

    # S'assurer que `axes` est un tableau (utile pour 1 seul graphe)
    if num_plots == 1:
        axes = np.array([axes])

    cat_labels = ["Unstim", "Stim True", "Stim Pred"]
    cat_colors = ["blue", "red", "green"]

    for i, (pt, ax) in enumerate(zip(pts_sorted, axes.flatten())):
        for label, color in zip(cat_labels, cat_colors):
            arr = dist_data[pt][label]
            if arr.size > 0:
                sns.kdeplot(arr, ax=ax, label=f"{label} (n={arr.size})", color=color, fill=True, alpha=0.3)

        ax.set_title(f"Patient: {pt}", fontsize=14)
        ax.set_xlabel("Value", fontsize=12)
        ax.set_ylabel("Density", fontsize=12)
        ax.legend(fontsize=10)
        ax.grid(True)

    # Supprimer les axes vides
    for j in range(i + 1, len(axes.flatten())):
        fig.delaxes(axes.flatten()[j])

    plt.savefig(out_file, dpi=200)
    plt.show()


In [ ]:
def plot_result(prediction_path,original_path,marker,outdir_path):
    target=ad.read(original_path)
    target1=target[:,marker].copy()
    stim=pd.DataFrame(target1[target1.obs['condition']=='stim'].X)
    unstim=pd.DataFrame(target1[target1.obs['condition']=='control'].X)

    dataf=pd.read_csv(prediction_path)
    dataf["Stim Pred"]=dataf[marker]
    dataf['Stim True']=stim.iloc[:,0]
    dataf['Unstim']=unstim.iloc[:,0]
    
    dist_data = {
    "Patient_1": {
        "Stim True": dataf["Stim True"].values,
        "Stim Pred": dataf["Stim Pred"].values,
        "Unstim": dataf["Unstim"].values}}

    create_density_plots2(dist_data, outdir_path, title_suffix="")     
    return dataf[['Stim True','Stim Pred','Unstim']]

In [22]:
EVALUATION_MARKERS = [
    '151Eu_pp38', '155Gd_pS6', '149Sm_pCREB', 
    '159Tb_pMAPKAPK2', '166Er_pNFkB', '167Er_pERK12'
]

train on CMC, pp38, pS6, LPS

In [30]:
plot_result('results/test_1/model-cellot/unseen_data/prediction_HCAA.csv','results/test_1/model-cellot/unseen_data/combined_LPS_HCAA.h5ad','149Sm_pCREB',"/Users/MacBook/stanford/cellot/plots/density_plot_test_sherlock3.png")

,Stim True,Stim Pred,Unstim
0,0.204033,0.016110,0.026968
1,0.034865,0.930722,0.965846
2,0.093312,-0.006110,0.007852
3,0.122988,-0.007951,0.000000
4,0.000000,0.631354,0.655327
...,...,...,...
11968,0.315942,-0.009274,0.000000
11969,0.415516,0.024459,0.031487
11970,0.000000,0.419385,0.497322
11971,0.000000,0.141170,0.158516
